In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import nltk
%matplotlib inline

import grequests # You will get errors if grequests is not above requests
import requests
from requests.adapters import HTTPAdapter, Retry
import concurrent.futures
from bs4 import BeautifulSoup
import lxml
import cchardet
import re

import pyinputplus as pyip
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import pickle
import random
import math 
import string

from unique_route_handling import *

%load_ext autoreload
%autoreload 2

tqdm.pandas()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

import warnings
warnings.simplefilter("ignore", category=UserWarning) # Grequests is a monkeypatch and not intended to be used with jupyter. This silences an annoying userwarning.

--- IMPORT ---

In [25]:
upload_link = 'https://www.mountainproject.com/user/200180658/brayden-l'
upload_type = 'todo'
df_ulist, _ = download_routelist(upload_type, upload_link)

--- DATA CLEANSE AND STANDARDIZE ---

In [26]:
df_ulist = data_standardize(df_ulist)

--- GRADE HOMOGENIZATION AND ROUTE LENGTH CLEANUP ---

In [27]:
df_ulist = route_length_fixer(df_ulist, 'express')

In [ ]:
grade_settings = ['letter', 'even_rand', 'flat', 'even_rand']
df_ulist = grade_homo(df_ulist, *grade_settings)

--- SCRAPE ---

In [ ]:
df_ulist = routescrape_syncro(df_ulist)

--- ANALYZE ---

In [ ]:
df_ulist = extract_tick_details(df_ulist, par=True)

In [ ]:
# Let's save the now scraped dataframe to a pickle file
df_ulist.to_pickle('../Data_Archive/df_todo_archive')

In [ ]:
# To load the new pickle file
picklefile = open('../Data_Archive/df_todo_archive', 'rb')
df_ulist = pickle.load(picklefile)

In [ ]:
df_ulist = tick_analysis(df_ulist)

--- SPLIT OUTPUT INTO BOULDER AND ROPED SUBFRAME ---

In [ ]:
# Select Boulder or Route
df_ulist_r = df_ulist[df_ulist['Route Type'] != 'Boulder']
df_ulist_b = df_ulist[df_ulist['Route Type'] == 'Boulder']

In [ ]:
# Filter

In [ ]:
min_ticks = 30

# Rarely led
df_low_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] < 0.4) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio')

# Rarely toproped
df_high_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] > 0.9) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio', ascending=False)

# Hard to OS
df_low_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')
df_low_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')

# High OS
df_high_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)
df_high_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)

# Find route on list that is hardest and easiest to OS given a grade
OS_cutoff_num = 3


In [ ]:
grouplist = df_ulist['Rating'].unique()
outlist = []
for group in grouplist:
    outlist.extend(list(df_ulist[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist['Rating'] == group)].nsmallest(10, 'OS Ratio').index))
df_ulist.loc[outlist].sort_values('OS Ratio')